<a href="https://colab.research.google.com/github/nzcbass/Pythontoopenai/blob/main/Add_Visa_date_to_job_Order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
from datetime import datetime, timedelta

# Load your DataFrames
file_path1 = '/content/Candidate List.csv'
df1 = pd.read_csv(file_path1)

file_path2 = '/content/Joborder.csv'
df2 = pd.read_csv(file_path2)

# First, let's ensure that both 'Candidate No.' and 'Candidate ID' are of the same type, usually string
df1['Candidate No'] = df1['Candidate No'].astype(str)
df2['Candidate ID'] = df2['Candidate ID'].astype(str)

# Now, create a mapping series from df1
mapping_series = df1.set_index('Candidate No')['Visa Expiry Date']

# Use the map function to create the new 'Visa Expiry Date' column in df2
df2['Visa Expiry Date'] = df2['Candidate ID'].map(mapping_series)

# Note: This will leave NaN in the new column where there are no matching 'Candidate ID'

# Remove the specified columns in df2
columns_to_remove = ['Position Type', 'Type', 'Parent Name', 'Client No.', 'Cost Centre ID', 'Cost Centre Name', ' Order by Contact', 'Site Contact', 'Order Date & Time']
df2 = df2.drop(columns=columns_to_remove)

# Check if 'End Date' and 'Visa Expiry' columns exist in df2
if 'End Date' in df2.columns and 'Visa Expiry Date' in df2.columns:
    # Get the index of 'End Date' column
    end_date_index = df2.columns.get_loc('End Date')

    # Reorder the columns
    cols = list(df2.columns)
    # Remove 'Visa expiry date' from its current position
    cols.remove('Visa Expiry Date')
    # Insert it right after 'End Date'
    cols.insert(end_date_index + 1, 'Visa Expiry Date')

    # Reindex the dataframe with the new column order
    df2 = df2[cols]
else:
    print("One or both of the columns 'End Date' and 'Visa expiry date' do not exist in df2")

# Display the first few rows to confirm the change
# print(df2.head())

# Ensure that 'End Date' and 'Visa Expiry Date' are in datetime format
df2['End Date'] = pd.to_datetime(df2['End Date'], dayfirst=True)
df2['Visa Expiry Date'] = pd.to_datetime(df2['Visa Expiry Date'], dayfirst=True)

# Update the function to handle all conditions
def determine_status(row):
    # Current date
    today = pd.Timestamp('now').normalize()

    # If Visa Expiry Date is provided and within 2 weeks of End Date
    if pd.notnull(row['Visa Expiry Date']) and (row['Visa Expiry Date'] - row['End Date']).days <= 14:
        return 'Extend Visa to extend job'
    # If End Date is in the past
    elif row['End Date'] < today:
        return 'Overdue'
    # If End Date is within 0-7 days
    elif row['End Date'] <= today + timedelta(days=7):
        return '< 7 days until job closes'
    # If End Date is within 8-14 days
    elif row['End Date'] <= today + timedelta(days=14):
        return '8-14 days until job closes'
    # If End Date is within 15-30 days
    elif row['End Date'] <= today + timedelta(days=30):
        return '15-30 days until job closes'
    # If none of the above, leave the status as None (or you can return an empty string or another placeholder)
    else:
        return None

# Apply the function to each row
df2['Status'] = df2.apply(determine_status, axis=1)

# Now df2 will have the 'Status' column with the appropriate statuses

# The new DataFrame df2 will have the 'Status' column with the 'Extend Visa' where appropriate

# Move the 'Status' column to the right of 'Visa Expiry Date'
# First store the 'Status' column
status = df2['Status']
# Then drop the 'Status' column from its current location
df2.drop(labels=['Status'], axis=1, inplace=True)
# Insert the 'Status' column after 'Visa Expiry Date'
visa_expiry_date_index = df2.columns.get_loc('Visa Expiry Date') + 1
df2.insert(visa_expiry_date_index, 'Status', status)

# Sort df2 by 'End Date' in ascending order (oldest dates on top)
df2.sort_values(by='End Date', inplace=True)

# Now df2 is sorted by 'End Date' with 'Status' column in the new location

# Drop the columns 'Candidate ID', 'Job No.', 'Filled Date', and 'Office'
df2.drop(columns=['Candidate ID', 'Job No.', 'Filled Date', 'Office'], inplace=True)


# Add a new column with empty strings or NaN for 'Comments for updates on jobs'
df2['Comments for updates on jobs'] = pd.NA  # or use "" for empty strings

# Sorting df2 by 'End Date' in ascending order
df2 = df2.sort_values(by='End Date', ascending=True)

# Define the file path for the new CSV file
output_file_path = '/content/Placed Jobs with Visa Expiry Date.csv'

# Save the df2 DataFrame to a CSV file
df2.to_csv(output_file_path, index=False)